In [11]:
"""
README: 

Before running:

1) Edit variable:
AUG_MODE = ['Up_All', 'Minority', 'RandomRotate',  'Majority_Down', RandomRotate]
based on your augmentation method.

2) Save and start running to avoid any err/exception
"""

"\nREADME: \n\nBefore running:\n\n1) Edit variable:\nAUG_MODE = ['Up_All', 'Minority', 'RandomRotate',  'Majority_Down', RandomRotate]\nbased on your augmentation method.\n\n2) Save and start running to avoid any err/exception\n"

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score,confusion_matrix
from itertools import product

In [13]:
import os 
#os.listdir(os.getcwd())

RES_DIR = './train_analysis/'

# MultiClass Result analysis

In [14]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools

py.init_notebook_mode(connected=True)
from itertools import product

In [15]:
CLASS = ['Norm', 'LSIL', 'HSIL', 'LSIL_HPV', 'SCC']

In [16]:
def visual_cmatrix(c_matrix, clazz=CLASS):
    trace = go.Heatmap(z=c_matrix,
                       x=clazz,
                       y=clazz
                       )

    layout = {'annotations':
              [
                {
                    "x": t[0],
                    "y": t[1],
                    "font": {"color":"white"},
                    "showarrow": False,
                    "text": str(c_matrix.T.flatten()[n]),
                    "xref": "x1",
                    "yref": "y1"
                } 
               for n, t in enumerate(product(clazz, clazz))
              ],
              'title': "Confusion Matrix",
              'xaxis': {"title": "Predicted Value"},
              'yaxis': {"title": "Real Value"}
             }

    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)

In [17]:
#for i in [1,2,3,4]:
#    print("Class {}: Accuracy: {}".format(CLASS[i], accuracy_score(gt_cat == i,preds_cat == i)))
#    print("Class {}: Precision: {}".format(CLASS[i], precision_score(gt_cat == i,preds_cat == i)))
#    print("Class {}: Recall: {}".format(CLASS[i], recall_score(gt_cat == i,preds_cat == i)))

In [18]:
#print(c_matrix)

In [19]:
#df = pd.read_csv(RES_DIR + DATASETS['TestSet'].format(MODELS[1]))

In [22]:
PADDING = "Warp" #{Double_ZoomOut_Padding, Padding, Warp}
FOLD_MODE = "CellBased" #{CellBased, FovBased}
DROPOUT = 0.5
CLF_MODE = "MultiClass" #{MultiClass, Binary}
#COLOR_MODE = "RGB"
COLOR_MODE = 'RGB'
#COLOR_MODE = ['HSV_CLAHE_V', 'HSV_CLAHE_S', 'HSV_CLAHE_SV', 'LAB_CLAHE_L', 'RGB',"HIST_BATCH_EQ"] 
#{HSV_CLAHE_V, HSV_CLAHE_S, HSV_CLAHE_SV, LAB_CLAHE_L, RGB}
AUG_MODE = ['Up_All']
#AUG_MODE = ['Minority', 'RandomRotate',  'Majority_Down']
#AUG_MODE = 'RandomRotate'

NomalLoss_MODELS = ['{}_{}_{}_{}_{}_{}_NormalLoss'.format(PADDING, aug_mode, FOLD_MODE, DROPOUT, CLF_MODE, COLOR_MODE) 
             for aug_mode in AUG_MODE]
WeightedLoss_MODELS = ['{}_{}_{}_{}_{}_{}_WeightedLoss'.format(PADDING, aug_mode, FOLD_MODE, DROPOUT, CLF_MODE, COLOR_MODE) 
              for aug_mode in AUG_MODE]

MODELS = NomalLoss_MODELS + WeightedLoss_MODELS
#MODELS = NomalLoss_MODELS
print(MODELS)
#MODELS = ['{}_{}_{}_{}_{}_{}'.format(PADDING, AUG_MODE, FOLD_MODE, DROPOUT, CLF_MODE, color_mode) 
#             for color_mode in COLOR_MODE]

DATASETS = {'TrainSet':"train_result_{}.csv",
            'ValidSet':"valid_result_{}.csv",
            'TestSet':"submission_{}.csv"}

#columns = ['DataSet', 'Model'] + list(product(CLASS, ['Precision', 'Recall', 'Accuracy']))
columns = [(' ', 'DataSet'), (' ', 'Model'), (' ', 'accuracy')] + \
          [(' ', 'b_accuracy'), (' ', 'b_precision'), (' ', 'b_recall')] + \
          list(product(CLASS, ['Precision', 'Recall']))
columns = pd.MultiIndex.from_tuples(columns)
table_multiclass = pd.DataFrame(columns=columns)

c_matrixes = []
for dataset, model in product(DATASETS.keys(), MODELS):
    #import pdb
    #pdb.set_trace()
    df = pd.read_csv(RES_DIR + DATASETS[dataset].format(model))
    gt_cat = df['gt'].copy()
    try:
        preds_cat = np.argmax(df.loc[:, 'preds_0':'preds_4'].values, axis=1)
    except:
        preds_cat = np.argmax(df.loc[:, 'positive_0':'positive_4'].values, axis=1)
    gt_cat[gt_cat == 5] = 4
    accuracy = accuracy_score(gt_cat, preds_cat)
    
    gt_b = (gt_cat > 0).astype(np.uint8)
    preds_b = (preds_cat > 0).astype(np.uint8)
    b_accuracy = accuracy_score(gt_b, preds_b)
    b_precision = precision_score(gt_b, preds_b)
    b_recall = recall_score(gt_b, preds_b)
    
    temp = [dataset, model, accuracy] + [b_accuracy, b_precision, b_recall]
    for clazz in range(len(CLASS)):
        precision = precision_score(gt_cat == clazz, preds_cat == clazz)
        recall = recall_score(gt_cat == clazz, preds_cat == clazz)
        #accuracy = accuracy_score(gt_cat == clazz, preds_cat == clazz)
        temp += [precision, recall]
        
    c_matrix = confusion_matrix(gt_cat, preds_cat)    
    c_matrixes.append(c_matrix)      
    table_multiclass = table_multiclass.append(dict(zip(columns, temp)), ignore_index=True)

['Warp_Up_All_CellBased_0.5_MultiClass_RGB_NormalLoss']


In [23]:
pd.set_option('display.max_colwidth', -1)
table_multiclass

\
    DataSet                                                Model  accuracy   
0  TestSet   Warp_Up_All_CellBased_0.5_MultiClass_RGB_NormalLoss  0.565217   
1  ValidSet  Warp_Up_All_CellBased_0.5_MultiClass_RGB_NormalLoss  0.600770   
2  TrainSet  Warp_Up_All_CellBased_0.5_MultiClass_RGB_NormalLoss  0.994865   

                                        Norm                LSIL            \
  b_accuracy b_precision  b_recall Precision    Recall Precision    Recall   
0  0.869565   0.903846    0.921569  0.764706  0.722222  0.545455  0.666667   
1  0.893453   0.915254    0.942408  0.825397  0.757282  0.592814  0.638710   
2  0.997433   0.997387    0.999127  0.997561  0.992718  1.000000  1.000000   

       HSIL            LSIL_HPV                 SCC            
  Precision    Recall Precision    Recall Precision    Recall  
0  0.371429  0.433333  0.384615  0.227273  0.695652  0.695652  
1  0.497297  0.550898  0.448598  0.387097  0.557252  0.574803  
2  0.994012  0.994012  0.997972  0.991935  0.982524  0.996063

In [ ]:
table_multiclass.to_csv('./multicalss.csv')

In [ ]:
visual_cmatrix(c_matrixes[-3])

In [ ]:
print(c_matrixes[0])

In [ ]:
def visual_mis_classified(true_class, pred_class, df):
    if type(true_class) == str:
        true_class = CLASS.index(true_class)
    if type(pred_class) == str:
        pred_class = CLASS.index(pred_class)
    #gt == 5 must change to 4 
    
    try:
        preds_cat = np.argmax(df.loc[:, 'preds_0':'preds_4'].values, axis=1)
    except:
        preds_cat = np.argmax(df.loc[:, 'positive_0':'positive_4'].values, axis=1)
   
    file_names = df.loc[(df['gt'] == true_class) & (preds_cat == pred_class), 'name']
    
    fig, axs = plt.subplots(nrows=len(file_names)//5+1, ncols=5, constrained_layout=False,figsize=(20,20))
    for i,ax in enumerate(axs.flatten()):
        if i<len(file_names):
            img = cv2.imread(file_names.iloc[i])
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            ax.imshow(img)
    return list(file_names)

In [ ]:
PADDING = "Warp" #{Double_ZoomOut_Padding, Padding, Warp}
FOLD_MODE = "CellBased" #{CellBased, FovBased}
DROPOUT = 0.5
CLF_MODE = "MultiClass" #{MultiClass, Binary}
COLOR_MODE = "RGB"
#COLOR_MODE = ['HSV_CLAHE_V', 'HSV_CLAHE_S', 'HSV_CLAHE_SV', 'LAB_CLAHE_L', 'RGB',"HIST_BATCH_EQ"] 
#{HSV_CLAHE_V, HSV_CLAHE_S, HSV_CLAHE_SV, LAB_CLAHE_L, RGB}
#AUG_MODE = ['Minority', 'RandomRotate', 'Majority_Down_Minority_Up', 'Majority_Down']
AUG_MODE = ['Up_All']
#AUG_MODE = 'RandomRotate'

MODELS = ['{}_{}_{}_{}_{}_{}'.format(PADDING, aug_mode, FOLD_MODE, DROPOUT, CLF_MODE, COLOR_MODE) 
             for aug_mode in AUG_MODE]

#MODELS = ['{}_{}_{}_{}_{}_{}'.format(PADDING, AUG_MODE, FOLD_MODE, DROPOUT, CLF_MODE, color_mode) 
#             for color_mode in COLOR_MODE]

DATASETS = {'TrainSet':"train_result_{}.csv",
            'ValidSet':"valid_result_{}.csv",
            'TestSet':"submission_{}.csv"}


_df = pd.read_csv(RES_DIR + DATASETS['TestSet'].format(MODELS[1]))
#_df.loc[_df['gt']==5,'gt'] = 4

In [ ]:
#file_name = visual_mis_classified('LSIL_HPV', 'Norm', df)

In [ ]:
#print(file_name)

In [ ]:
#import glob